# Création d'un client MCP local avec LlamaIndex

Ce notebook Jupyter vous guide dans la création d'un **client MCP (Model Context Protocol) local** capable de communiquer avec une base de données via les outils exposés par un serveur MCP, le tout sur votre machine. 

In [15]:
import nest_asyncio
nest_asyncio.apply()

## 2  Configurer un LLM local

In [16]:
from llama_index.llms.ollama import Ollama
from llama_index.core import Settings

llm = Ollama(model="llama3.2", request_timeout=120.0)
Settings.llm = llm

## 3  Initialisez le client MCP et créez l'agent

Pointez le client vers le **point de terminaison SSE** de votre serveur MCP local (par défaut, voir ci-dessous) et répertoriez les outils disponibles.

In [17]:
from llama_index.tools.mcp import BasicMCPClient, McpToolSpec

mcp_client = BasicMCPClient("http://127.0.0.1:8000/sse")
mcp_tools = McpToolSpec(client=mcp_client) # Vous pouvez également passer une liste des outils autorisés.

In [18]:
tools = await mcp_tools.to_tool_list_async()
for tool in tools:
    print(tool.metadata.name, tool.metadata.description)

add_data Add new data to the people table using a SQL INSERT query.

    Args:
        query (str): SQL INSERT query following this format:
            INSERT INTO people (name, age, profession)
            VALUES ('John Doe', 30, 'Engineer')
        
    Schema:
        - name: Text field (required)
        - age: Integer field (required)
        - profession: Text field (required)
        Note: 'id' field is auto-generated
    
    Returns:
        bool: True if data was added successfully, False otherwise
    
    Example:
        >>> query = '''
        ... INSERT INTO people (name, age, profession)
        ... VALUES ('Alice Smith', 25, 'Developer')
        ... '''
        >>> add_data(query)
        True
    
read_data Read data from the people table using a SQL SELECT query.

    Args:
        query (str, optional): SQL SELECT query. Defaults to "SELECT * FROM people".
            Examples:
            - "SELECT * FROM people"
            - "SELECT name, age FROM people WHERE ag

## 3  Définir l'invite système
Cette invite guide le LLM lorsqu'il doit décider comment et quand appeler des outils.

In [19]:
SYSTEM_PROMPT = """\
Tu es un assistant IA pour Tool Calling.

Avant d'aider un utilisateur, Tu dois utiliser des outils pour interagir avec notre base de données.
"""

## 4  Fonction d'aide : `get_agent()`
Crée une `FunctionAgent` connecté à la liste d'outils MCP et au LLM de votre choix.

In [20]:
from llama_index.tools.mcp import McpToolSpec
from llama_index.core.agent.workflow import FunctionAgent

async def get_agent(tools: McpToolSpec):
    tools = await tools.to_tool_list_async()
    agent = FunctionAgent(
        name="Agent",
        description="Un agent capable de fonctionner avec notre logiciel de base de données.",
        tools=tools,
        llm=Ollama(model="llama3.2"),
        system_prompt=SYSTEM_PROMPT,
    )
    return agent

## 5  Fonction d'aide : `handle_user_message()`
Transmet les appels d'outils intermédiaires (pour plus de transparence) et renvoie la réponse finale.

In [21]:
from llama_index.core.agent.workflow import (
    FunctionAgent, 
    ToolCallResult, 
    ToolCall)

from llama_index.core.workflow import Context

async def handle_user_message(
    message_content: str,
    agent: FunctionAgent,
    agent_context: Context,
    verbose: bool = False,
):
    handler = agent.run(message_content, ctx=agent_context)
    async for event in handler.stream_events():
        if verbose and type(event) == ToolCall:
            print(f"Calling tool {event.tool_name} with kwargs {event.tool_kwargs}")
        elif verbose and type(event) == ToolCallResult:
            print(f"Tool {event.tool_name} returned {event.tool_output}")

    response = await handler
    return str(response)

## 6  Initialisez le client MCP et créez l'agent
Dirigez le client vers le **point de terminaison SSE** de votre serveur MCP local (valeur par défaut indiquée ci-dessous), créez l'agent et configurez le contexte de l'agent.

In [22]:
from llama_index.tools.mcp import BasicMCPClient, McpToolSpec


mcp_client = BasicMCPClient("http://127.0.0.1:8000/sse")
mcp_tool = McpToolSpec(client=mcp_client)

# get the agent
agent = await get_agent(mcp_tool)

# create the agent context
agent_context = Context(agent)

In [ ]:
# Run the agent!
while True:
    user_input = input("Entrez votre message: ")
    if user_input == "sortir":
        break
    print("User: ", user_input)
    response = await handle_user_message(user_input, agent, agent_context, verbose=True)
    print("Agent: ", response)